In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
BASE_PATH = "/content/drive/MyDrive/mens-march-mania-2022/MDataFiles_Stage2/"
RESULTS_NAME = "MNCAATourneyDetailedResults.csv"
SEED_NAME = "MNCAATourneySeeds.csv"
RANK_NAME = "MMasseyOrdinals_thruDay128.csv"
TEST_NAME = "MSampleSubmissionStage2.csv"

model_type = 'Autolgbm'  
message='baseline'

train_filename = "./train.csv"
output = "output"
test_filename = "./test.csv"
task = None
targets = ['target']
features = None
categorical_features = None
use_gpu = False
num_folds = 10
seed = 42
num_trials = 10000
time_limit = 600
fast = False

TEST = False

In [ ]:
!pip install autolgbm
!pip install jaxlib
!pip install gpjax
!pip install multipledispatch
!pip install optax



In [ ]:
import pandas as pd
import numpy as np
import jax
import jax.numpy as jnp
import jax.random as jr
import random
import time
import gc
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
from lightgbm import LGBMRegressor
import matplotlib.pylab as plt
import seaborn as sns
from autolgbm import AutoLGBM

In [ ]:
data1=pd.read_csv('/content/drive/MyDrive/mens-march-mania-2022/MDataFiles_Stage2/MNCAATourneySeedRoundSlots.csv')
data2=pd.read_csv('/content/drive/MyDrive/mens-march-mania-2022/MDataFiles_Stage2/MNCAATourneySeeds.csv')
data2 = data2 [(data2 ["Season"] == 2022)]
data2= data2[['Seed', 'TeamID']]
data3=pd.read_csv('/content/drive/MyDrive/mens-march-mania-2022/MDataFiles_Stage2/MNCAATourneySlots.csv')
data4=pd.read_csv('/content/drive/MyDrive/mens-march-mania-2022/MDataFiles_Stage2/MTeams.csv')
data3 = data3 [(data3 ["Season"] == 2022)]
data5=pd.read_csv('/content/drive/MyDrive/mens-march-mania-2022/MDataFiles_Stage2/MTeamConferences.csv')
data5 = data5 [(data5 ["Season"] == 2022)]
data6=pd.read_csv('/content/drive/MyDrive/mens-march-mania-2022/MDataFiles_Stage2/MMasseyOrdinals_thruDay128.csv')
data6 = data6 [(data6 ["Season"] == 2022)]
data6 = data6 [(data6 ["RankingDayNum"] > 122)]
data6 = data6 [(data6 ["SystemName"] == 'NET')]
data7=pd.read_csv('/content/drive/MyDrive/mens-march-mania-2022/MDataFiles_Stage2/MRegularSeasonCompactResults.csv')
data7 = data7 [(data7 ["Season"] == 2022)]
data9=pd.read_csv('/content/drive/MyDrive/mens-march-mania-2022/MDataFiles_Stage2/MRegularSeasonCompactResults.csv')
data9 = data9 [(data9 ["Season"] == 2022)]
data8=pd.read_csv('/content/drive/MyDrive/mens-march-mania-2022/MDataFiles_Stage2/MRegularSeasonDetailedResults.csv')
data8 = data8 [(data8 ["Season"] == 2022)]

In [ ]:
data1.head()

In [ ]:
data3.head()

In [ ]:
output4 = pd.merge(data2, data4, on='TeamID', how='inner')
output4=pd.merge(output4,data5, on='TeamID', how='inner')
output4=pd.merge(output4,data6, on='TeamID', how='inner')
output4= output4[['Seed', 'TeamID', 'TeamName','ConfAbbrev','OrdinalRank']]
output4.head()

In [ ]:
data7.sort_values(data7.columns[1],axis=0,inplace=True)
data7 = data7[['DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore']]
data7.head()

In [ ]:
#data9.sort_values(data9.columns[3],axis=0,inplace=True)
data9 = data9[['DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore']]
data9.head()

In [ ]:
df_TDresults = pd.read_csv(BASE_PATH + RESULTS_NAME)
df_TDresults.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,134,1421,92,1411,84,N,1,32,69,...,31,14,31,17,28,16,15,5,0,22
1,2003,136,1112,80,1436,51,N,0,31,66,...,16,7,7,8,26,12,17,10,3,15
2,2003,136,1113,84,1272,71,N,0,31,59,...,28,14,21,20,22,11,12,2,5,18
3,2003,136,1141,79,1166,73,N,0,29,53,...,17,12,17,14,17,20,21,6,6,21
4,2003,136,1143,76,1301,74,N,1,27,64,...,21,15,20,10,26,16,14,5,8,19


In [ ]:
df_TDresults = pd.read_csv(BASE_PATH + RESULTS_NAME)
df_TDresults.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,134,1421,92,1411,84,N,1,32,69,...,31,14,31,17,28,16,15,5,0,22
1,2003,136,1112,80,1436,51,N,0,31,66,...,16,7,7,8,26,12,17,10,3,15
2,2003,136,1113,84,1272,71,N,0,31,59,...,28,14,21,20,22,11,12,2,5,18
3,2003,136,1141,79,1166,73,N,0,29,53,...,17,12,17,14,17,20,21,6,6,21
4,2003,136,1143,76,1301,74,N,1,27,64,...,21,15,20,10,26,16,14,5,8,19


In [ ]:
box_col = ["FGM","FGA","FGM3","FTM","FTA","OR","DR","Ast","TO","Stl","Blk","PF"]
df_boxW = df_TDresults[["WTeamID"]+["W" + col for col in box_col]]
df_boxL = df_TDresults[["LTeamID"]+["L" + col for col in box_col]]
df_boxW = df_boxW.rename(columns={"WTeamID":"TeamID"})
df_boxW = df_boxW.rename(columns={("W"+ col):col for col in box_col})
df_boxL = df_boxL.rename(columns={"LTeamID":"TeamID"})
df_boxL = df_boxL.rename(columns={("L"+ col):col for col in box_col})
df_box = pd.merge(df_boxW,df_boxL,on = ["TeamID"]+box_col,how="outer")
df_box = df_box.groupby(["TeamID"])[box_col].agg(np.mean).reset_index()
df_box.head()

,TeamID,FGM,FGA,FGM3,FTM,FTA,OR,DR,Ast,TO,Stl,Blk,PF
0,1101,18.000000,59.000000,4.000000,8.000000,11.000000,7.666667,18.000000,10.333333,9.333333,7.666667,1.666667,15.333333
1,1102,19.000000,40.000000,10.500000,12.000000,15.500000,1.500000,16.500000,15.000000,13.500000,7.000000,1.500000,14.000000
2,1103,19.666667,54.666667,6.000000,8.666667,14.333333,9.333333,19.666667,10.000000,12.666667,2.666667,3.666667,18.666667
3,1104,24.857143,52.714286,7.214286,15.428571,22.071429,10.428571,22.214286,13.214286,12.285714,5.571429,4.142857,16.714286
4,1105,23.000000,59.000000,10.000000,13.000000,18.000000,10.000000,19.000000,16.000000,14.000000,6.000000,3.000000,17.000000


In [ ]:
df_TDresults2 = df_TDresults
df_TDresults = df_TDresults.rename(columns={"WTeamID":"Team1ID","LTeamID":"Team2ID","WScore":"T1Score","LScore":"T2Score"})
df_TDresults = df_TDresults.rename(columns={f"W{col}":f"T1{col}" for col in box_col})
df_TDresults = df_TDresults.rename(columns={f"L{col}":f"T2{col}" for col in box_col})
df_TDresults2 = df_TDresults2.rename(columns={"WTeamID":"Team2ID","LTeamID":"Team1ID","WScore":"T2Score","LScore":"T1Score"})

In [ ]:
features = ["Season","Team1ID","Team2ID","T1Score","T2Score",'target']
df_TDresults['target'] = 1.0
df_TDresults2['target'] = 0.0
train = pd.merge(df_TDresults,df_TDresults2,on = features,how="outer")
train = train[features]
train.head()

,Season,Team1ID,Team2ID,T1Score,T2Score,target
0,2003,1421,1411,92,84,1.0
1,2003,1112,1436,80,51,1.0
2,2003,1113,1272,84,71,1.0
3,2003,1141,1166,79,73,1.0
4,2003,1143,1301,76,74,1.0


In [ ]:
box_T1 = df_box.copy()
box_T2 = df_box.copy()
box_T1.columns = ['Team1ID'] + ["T1"+col+"_mean" for col in box_col]
box_T2.columns = ['Team2ID'] + ["T2"+col+"_mean" for col in box_col]
train = pd.merge(train,box_T1,on = ["Team1ID"],how = "left")
train = pd.merge(train,box_T2,on = ["Team2ID"],how = "left")
train.head()

,Season,Team1ID,Team2ID,T1Score,T2Score,target,T1FGM_mean,T1FGA_mean,T1FGM3_mean,T1FTM_mean,...,T2FGM3_mean,T2FTM_mean,T2FTA_mean,T2OR_mean,T2DR_mean,T2Ast_mean,T2TO_mean,T2Stl_mean,T2Blk_mean,T2PF_mean
0,2003,1421,1411,92,84,1.0,23.500000,57.666667,6.166667,14.500000,...,6.125000,16.375000,23.250000,10.375000,20.625000,9.875000,11.250000,5.750000,2.750000,18.500000
1,2003,1112,1436,80,51,1.0,27.027778,56.111111,5.972222,16.111111,...,7.444444,7.777778,11.444444,12.333333,21.000000,12.000000,12.555556,6.222222,1.555556,17.555556
2,2003,1113,1272,84,71,1.0,24.375000,54.125000,7.250000,16.750000,...,6.000000,14.615385,22.307692,12.961538,22.576923,13.307692,11.423077,7.076923,4.730769,18.384615
3,2003,1141,1166,79,73,1.0,25.000000,55.000000,5.500000,14.000000,...,6.785714,9.571429,13.714286,9.214286,22.785714,13.357143,10.928571,4.928571,2.428571,17.071429
4,2003,1143,1301,76,74,1.0,23.600000,55.000000,5.000000,11.600000,...,6.333333,13.388889,19.222222,10.111111,24.500000,12.111111,11.500000,5.333333,3.722222,18.388889


In [ ]:
df_seeds = pd.read_csv(BASE_PATH + SEED_NAME)
df_seeds.head()

,Season,Seed,TeamID
0,1985,W01,1207
1,1985,W02,1210
2,1985,W03,1228
3,1985,W04,1260
4,1985,W05,1374


In [ ]:
df_seeds["seed"] = df_seeds['Seed'].apply(lambda x: int(x[1:3]))
df_seeds = df_seeds.groupby(["TeamID"])["seed"].agg(np.mean).reset_index()
seeds_T1 = df_seeds[['TeamID','seed']].copy()
seeds_T2 = df_seeds[['TeamID','seed']].copy()
seeds_T1.columns = ['Team1ID','T1_seed']
seeds_T2.columns = ['Team2ID','T2_seed']
train = pd.merge(train,seeds_T1,on = ["Team1ID"],how = "left")
train = pd.merge(train,seeds_T2,on = ["Team2ID"],how = "left")
train["seeddiff"] = train["T1_seed"] - train["T2_seed"]
train.head()

,Season,Team1ID,Team2ID,T1Score,T2Score,target,T1FGM_mean,T1FGA_mean,T1FGM3_mean,T1FTM_mean,...,T2OR_mean,T2DR_mean,T2Ast_mean,T2TO_mean,T2Stl_mean,T2Blk_mean,T2PF_mean,T1_seed,T2_seed,seeddiff
0,2003,1421,1411,92,84,1.0,23.500000,57.666667,6.166667,14.500000,...,10.375000,20.625000,9.875000,11.250000,5.750000,2.750000,18.500000,15.750000,15.500000,0.250000
1,2003,1112,1436,80,51,1.0,27.027778,56.111111,5.972222,16.111111,...,12.333333,21.000000,12.000000,12.555556,6.222222,1.555556,17.555556,4.333333,14.375000,-10.041667
2,2003,1113,1272,84,71,1.0,24.375000,54.125000,7.250000,16.750000,...,12.961538,22.576923,13.307692,11.423077,7.076923,4.730769,18.384615,8.714286,5.736842,2.977444
3,2003,1141,1166,79,73,1.0,25.000000,55.000000,5.500000,14.000000,...,9.214286,22.785714,13.357143,10.928571,4.928571,2.428571,17.071429,12.000000,8.687500,3.312500
4,2003,1143,1301,76,74,1.0,23.600000,55.000000,5.000000,11.600000,...,10.111111,24.500000,12.111111,11.500000,5.333333,3.722222,18.388889,7.785714,7.562500,0.223214


In [ ]:
df_MMOrdinals = pd.read_csv(BASE_PATH + RANK_NAME)
display(df_MMOrdinals.head())
df_rank = df_MMOrdinals.groupby(["TeamID"])["OrdinalRank"].agg(np.mean).reset_index()
df_rank.head()

,Season,RankingDayNum,SystemName,TeamID,OrdinalRank
0,2003,35,SEL,1102,159
1,2003,35,SEL,1103,229
2,2003,35,SEL,1104,12
3,2003,35,SEL,1105,314
4,2003,35,SEL,1106,260


,TeamID,OrdinalRank
0,1101,236.700207
1,1102,182.605110
2,1103,111.976844
3,1104,61.420144
4,1105,318.189167


In [ ]:
ranks_T1 = df_rank.copy()
ranks_T2 = df_rank.copy()
ranks_T1.columns = ['Team1ID','T1_rank_mean']
ranks_T2.columns = ['Team2ID','T2_rank_mean']
train = pd.merge(train,ranks_T1,on = ["Team1ID"],how = "left")
train = pd.merge(train,ranks_T2,on = ["Team2ID"],how = "left")
train["rankdiff"] = train["T1_rank_mean"] - train["T2_rank_mean"]
train.head()

,Season,Team1ID,Team2ID,T1Score,T2Score,target,T1FGM_mean,T1FGA_mean,T1FGM3_mean,T1FTM_mean,...,T2TO_mean,T2Stl_mean,T2Blk_mean,T2PF_mean,T1_seed,T2_seed,seeddiff,T1_rank_mean,T2_rank_mean,rankdiff
0,2003,1421,1411,92,84,1.0,23.500000,57.666667,6.166667,14.500000,...,11.250000,5.750000,2.750000,18.500000,15.750000,15.500000,0.250000,218.287574,243.170600,-24.883026
1,2003,1112,1436,80,51,1.0,27.027778,56.111111,5.972222,16.111111,...,12.555556,6.222222,1.555556,17.555556,4.333333,14.375000,-10.041667,26.639335,125.048969,-98.409634
2,2003,1113,1272,84,71,1.0,24.375000,54.125000,7.250000,16.750000,...,11.423077,7.076923,4.730769,18.384615,8.714286,5.736842,2.977444,85.243187,55.833211,29.409975
3,2003,1141,1166,79,73,1.0,25.000000,55.000000,5.500000,14.000000,...,10.928571,4.928571,2.428571,17.071429,12.000000,8.687500,3.312500,208.736582,48.500221,160.236361
4,2003,1143,1301,76,74,1.0,23.600000,55.000000,5.000000,11.600000,...,11.500000,5.333333,3.722222,18.388889,7.785714,7.562500,0.223214,93.525500,63.609165,29.916335


In [ ]:
train = train.drop(columns = ['T1Score','T2Score'])
train.head()

,Season,Team1ID,Team2ID,target,T1FGM_mean,T1FGA_mean,T1FGM3_mean,T1FTM_mean,T1FTA_mean,T1OR_mean,...,T2TO_mean,T2Stl_mean,T2Blk_mean,T2PF_mean,T1_seed,T2_seed,seeddiff,T1_rank_mean,T2_rank_mean,rankdiff
0,2003,1421,1411,1.0,23.500000,57.666667,6.166667,14.500000,19.000000,10.666667,...,11.250000,5.750000,2.750000,18.500000,15.750000,15.500000,0.250000,218.287574,243.170600,-24.883026
1,2003,1112,1436,1.0,27.027778,56.111111,5.972222,16.111111,20.444444,10.805556,...,12.555556,6.222222,1.555556,17.555556,4.333333,14.375000,-10.041667,26.639335,125.048969,-98.409634
2,2003,1113,1272,1.0,24.375000,54.125000,7.250000,16.750000,23.375000,8.500000,...,11.423077,7.076923,4.730769,18.384615,8.714286,5.736842,2.977444,85.243187,55.833211,29.409975
3,2003,1141,1166,1.0,25.000000,55.000000,5.500000,14.000000,19.500000,12.500000,...,10.928571,4.928571,2.428571,17.071429,12.000000,8.687500,3.312500,208.736582,48.500221,160.236361
4,2003,1143,1301,1.0,23.600000,55.000000,5.000000,11.600000,17.700000,10.800000,...,11.500000,5.333333,3.722222,18.388889,7.785714,7.562500,0.223214,93.525500,63.609165,29.916335


In [ ]:
test = pd.read_csv(BASE_PATH + TEST_NAME)
test.head()

,ID,Pred
0,2022_1103_1104,0.5
1,2022_1103_1112,0.5
2,2022_1103_1116,0.5
3,2022_1103_1120,0.5
4,2022_1103_1124,0.5


In [ ]:
test["Season"] = test['ID'].apply(lambda x: int(x[0:4]))
test["Team1ID"] = test['ID'].apply(lambda x: int(x[5:9]))
test["Team2ID"] = test['ID'].apply(lambda x: int(x[10:14]))
test.head()

,ID,Pred,Season,Team1ID,Team2ID
0,2022_1103_1104,0.5,2022,1103,1104
1,2022_1103_1112,0.5,2022,1103,1112
2,2022_1103_1116,0.5,2022,1103,1116
3,2022_1103_1120,0.5,2022,1103,1120
4,2022_1103_1124,0.5,2022,1103,1124


In [ ]:
test = pd.merge(test,box_T1,on = ["Team1ID"],how = "left")
test = pd.merge(test,box_T2,on = ["Team2ID"],how = "left")
test.head()

,ID,Pred,Season,Team1ID,Team2ID,T1FGM_mean,T1FGA_mean,T1FGM3_mean,T1FTM_mean,T1FTA_mean,...,T2FGM3_mean,T2FTM_mean,T2FTA_mean,T2OR_mean,T2DR_mean,T2Ast_mean,T2TO_mean,T2Stl_mean,T2Blk_mean,T2PF_mean
0,2022_1103_1104,0.5,2022,1103,1104,19.666667,54.666667,6.0,8.666667,14.333333,...,7.214286,15.428571,22.071429,10.428571,22.214286,13.214286,12.285714,5.571429,4.142857,16.714286
1,2022_1103_1112,0.5,2022,1103,1112,19.666667,54.666667,6.0,8.666667,14.333333,...,5.972222,16.111111,20.444444,10.805556,24.333333,13.833333,11.694444,6.333333,3.888889,17.000000
2,2022_1103_1116,0.5,2022,1103,1116,19.666667,54.666667,6.0,8.666667,14.333333,...,4.384615,16.000000,21.615385,10.692308,22.153846,11.615385,12.000000,6.538462,3.846154,17.307692
3,2022_1103_1120,0.5,2022,1103,1120,19.666667,54.666667,6.0,8.666667,14.333333,...,9.100000,12.800000,18.800000,10.200000,23.400000,12.400000,10.600000,7.100000,4.400000,18.500000
4,2022_1103_1124,0.5,2022,1103,1124,19.666667,54.666667,6.0,8.666667,14.333333,...,7.000000,13.360000,17.800000,11.920000,22.520000,14.800000,10.560000,7.040000,3.360000,17.120000


In [ ]:
test = pd.merge(test,seeds_T1,on = ["Team1ID"],how = "left")
test = pd.merge(test,seeds_T2,on = ["Team2ID"],how = "left")
test["seeddiff"] = test["T1_seed"] - test["T2_seed"]

In [ ]:
test = pd.merge(test,ranks_T1,on = ["Team1ID"],how = "left")
test = pd.merge(test,ranks_T2,on = ["Team2ID"],how = "left")
test["rankdiff"] = test["T1_rank_mean"] - test["T2_rank_mean"]

In [ ]:
test = test.drop(columns = ['ID','Pred'])
test.head()

,Season,Team1ID,Team2ID,T1FGM_mean,T1FGA_mean,T1FGM3_mean,T1FTM_mean,T1FTA_mean,T1OR_mean,T1DR_mean,...,T2TO_mean,T2Stl_mean,T2Blk_mean,T2PF_mean,T1_seed,T2_seed,seeddiff,T1_rank_mean,T2_rank_mean,rankdiff
0,2022,1103,1104,19.666667,54.666667,6.0,8.666667,14.333333,9.333333,19.666667,...,12.285714,5.571429,4.142857,16.714286,13.6,6.166667,7.433333,111.976844,61.420144,50.556700
1,2022,1103,1112,19.666667,54.666667,6.0,8.666667,14.333333,9.333333,19.666667,...,11.694444,6.333333,3.888889,17.000000,13.6,4.333333,9.266667,111.976844,26.639335,85.337510
2,2022,1103,1116,19.666667,54.666667,6.0,8.666667,14.333333,9.333333,19.666667,...,12.000000,6.538462,3.846154,17.307692,13.6,6.181818,7.418182,111.976844,61.044654,50.932191
3,2022,1103,1120,19.666667,54.666667,6.0,8.666667,14.333333,9.333333,19.666667,...,10.600000,7.100000,4.400000,18.500000,13.6,6.400000,7.200000,111.976844,96.834078,15.142767
4,2022,1103,1124,19.666667,54.666667,6.0,8.666667,14.333333,9.333333,19.666667,...,10.560000,7.040000,3.360000,17.120000,13.6,4.818182,8.781818,111.976844,49.867535,62.109310


In [ ]:
train.to_csv("train.csv",index = None)
test.to_csv("test.csv",index = None)

In [ ]:
features=None

In [ ]:
algbm = AutoLGBM(
    train_filename=train_filename,
    output=output,
    test_filename=test_filename,
    task=task,
    targets=targets,
    features=features,
    categorical_features=categorical_features,
    use_gpu=use_gpu,
    num_folds=num_folds,
    seed=seed,
    num_trials=num_trials,
    time_limit=time_limit,
    fast=fast,
)


2022-04-24 14:30:57.660 | INFO     | autolgbm.autolgbm:__post_init__:42 - Output directory: output


In [ ]:
algbm.train()

In [ ]:
submission = pd.read_csv("/content/drive/MyDrive/mens-march-mania-2022/MDataFiles_Stage2/MSampleSubmissionStage2.csv")
autolgb_pred = pd.read_csv("/content/output/test_predictions.csv")
submission['Pred'] = autolgb_pred['1.0']
# submission.rename(columns = {'1.0':'Pred'}, inplace = True)
submission.to_csv("submission.csv", index=False)
submission.head()

,ID,Pred
0,2022_1103_1104,0.070630
1,2022_1103_1112,0.065156
2,2022_1103_1116,0.087231
3,2022_1103_1120,0.072933
4,2022_1103_1124,0.064565


In [ ]:
import math
import jax.numpy as jnp
from jax.experimental import stax
import jax
from jax import random
from jax import grad, jit, vmap
from jax.experimental import optimizers
## LOSS FUNCTION ##

def loss(params, apply_fn, inputs, targets):
    preds = apply_fn(params, inputs)
    out = np.mean((preds - jax.nn.one_hot(targets, 2))**2)
    print(out)
    return out

In [ ]:
teamname = pd.read_csv("/content/drive/MyDrive/mens-march-mania-2022/MDataFiles_Stage2/MTeams.csv")
probs = pd.read_csv("/content/submission.csv")
teamprob = {}
for Team in teamname["TeamID"]:
  prob = 0
  i=0
  totalvs = 0
  for vs in probs["ID"]:
    x = vs.split("_")
    strteam = str(Team)
    if (strteam in x[1]):
      totalvs+=1
      prob += probs["Pred"][i]
    elif(strteam in x[2]):
      totalvs+=1
      prob += (1-probs["Pred"][i])
    i+=1
  if totalvs>0:
    teamprob[Team] = prob/totalvs

In [ ]:
sort_prob = sorted(teamprob.items(), key=lambda x: x[1], reverse = True )
top5 = sort_prob[:5]
print(sort_prob)
for tup in top5:
  i=0
  for Team in teamname["TeamID"]:
    if Team == tup[0]:
      print(Team, teamname["TeamName"][i],tup[1])
    i+=1

[(1181, 0.7818805013567883), (1314, 0.7659462933891015), (1163, 0.7657057265004449), (1246, 0.7643922426023472), (1242, 0.7598225310091599), (1276, 0.7570326251315845), (1437, 0.7471983415125518), (1112, 0.7372296218867722), (1326, 0.7359234925717001), (1277, 0.7262321427201311), (1211, 0.7204105843036864), (1124, 0.709078102588707), (1458, 0.6893246205090945), (1417, 0.6838907285159909), (1397, 0.6708521437657909), (1272, 0.6669106281557101), (1345, 0.6667986836629888), (1403, 0.6550399359665178), (1274, 0.6430791202313895), (1266, 0.6427603179126443), (1104, 0.6382763966482543), (1400, 0.6239217844688006), (1235, 0.6193223196438699), (1361, 0.6184369456470277), (1120, 0.6182497231504682), (1323, 0.6173453789341158), (1425, 0.6173388000485808), (1231, 0.6076948292986556), (1222, 0.59658848763455), (1228, 0.5946272458264384), (1261, 0.5945148706296907), (1116, 0.5920884747170975), (1260, 0.5841823659976747), (1234, 0.5607838630548333), (1371, 0.555744569576371), (1388, 0.55513594579436